In [1]:
import os
import cv2
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model, Model, model_from_json, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, InputLayer, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, GlobalAveragePooling2D

from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.losses import mean_squared_error

from load_data_metrics import load_data, data_balanced, metrics

# Model

In [2]:
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in resnet.layers:
    layer.trainable = False
output = resnet.layers[-1].output
output = Flatten()(output)
resnet_model = Model(resnet.input, output)

# Load dataset

In [3]:
def load_path(path):
    labels_ = os.listdir(path)
    features = []
    labels   = []
    for i, label in enumerate(labels_):
        cur_path = path + "/" + label 
        for image_path in glob.glob(cur_path + "/*"):
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            feature = resnet_model.predict(x)
            flat = feature.flatten()
            features.append(flat)
            labels.append(label)
            
        print ("[INFO] completed label - " + label)
    return features, labels

data, labels = load_data(load_path)


[INFO] completed label - 0_Normal
[INFO] completed label - 1_Glaucoma
[INFO] completed label - 0_Normal
[INFO] completed label - 1_Glaucoma
[INFO] completed label - 0_Normal
[INFO] completed label - 1_Glaucoma
[INFO] completed label - 0_Normal
[INFO] completed label - 1_Glaucoma
[INFO] completed label - 0_Normal
[INFO] completed label - 1_Glaucoma
Data:  (2390, 100352)
Normal:  1702
Glaucoma:  688


# Training model

In [ ]:
def balanced_method(data_comp, labels_comp, method):

    if method=='None':
        data = data_comp
        labels = labels_comp
    elif method=='random':
        data, labels = data_balanced(data_comp, labels_comp, 'random')
    elif method=='nm':
        data, labels = data_balanced(data_comp, labels_comp, 'nm')
    elif method=='cluster':
        data, labels = data_balanced(data_comp, labels_comp, 'cluster')
    
    
    kfold = StratifiedKFold(n_splits=10, shuffle = True)
    
    for train_ix, test_ix in kfold.split(data, labels):
        
        X_train, X_test = data[train_ix], data[test_ix]
        y_train, y_test = labels[train_ix], labels[test_ix]

        
        y_train = to_categorical(y_train)
        y_test = to_categorical(y_test)
        
        
        scaler = preprocessing.StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train) 
        X_test = scaler.transform(X_test) 

        input_shape = resnet_model.output_shape[1]

        model_transfer = Sequential()
        model_transfer.add(InputLayer(input_shape=(input_shape,)))  
        model_transfer.add(Dense(512, activation='relu', input_dim=input_shape))
        model_transfer.add(Dropout(0.3))
        model_transfer.add(Dense(512, activation='relu'))
        model_transfer.add(Dropout(0.3))
        model_transfer.add(Dense(2, activation='sigmoid'))


        model_transfer.compile(loss='mse', optimizer='sgd',
                      metrics=['accuracy'])

        history = model_transfer.fit(X_train, y_train, batch_size=32, 
                                     epochs=100, verbose=0, shuffle=True)
        
        pred = np.round(model_transfer.predict(X_test),0)
        y_pred = pred[:,1]
        y_true = y_test[:,1]

        acuracia, precision, sensitivity, specificity, f_score, tp, tn, fp, fn = metrics(y_pred, y_true)
        
    return


In [ ]:
# Unbalanced
unbalanced = balanced_method(data, labels, 'None')

In [ ]:
# Near miss 
nm_method = balanced_method(data, labels, 'nm')

In [ ]:
# Cluster centroid 
cluster_method = balanced_method(data, labels, 'cluster')

In [ ]:
# Random 
random_method = balanced_method(data, labels, 'random')